In [96]:
# imports
import requests
import pandas as pd
import numpy as np
import json
import os
from dotenv import load_dotenv

df_stations = pd.read_csv('df_stations.csv')[['id','name','total_bikes','latitude','longitude']]
#df_stations['ll'] = df_stations['latitude'].astype(str) + ',' + df_stations['longitude'].astype(str)

In [97]:
df_stations

,id,name,total_bikes,latitude,longitude
0,d334f65221f6a681f8b03433f36e2075,T19 Kiwanis Park,9,33.366488,-111.935612
1,cac663c57dee1dafd5f152bb906e97d5,P211 10th St / Pierce,6,33.455654,-112.060592
2,8d29ea89964d1eb73171310f718fe7e5,P196 2nd St. and Roosevelt,9,33.458995,-112.071128
3,6628d10945695a8a19a0ede32f949bc4,T17 Tempe Beach Park,23,33.429720,-111.940602
4,b953cdb6c1721e12c870e32cb780998c,ASU Goldwater Ctr. for Science - Tyler Mall (V...,6,33.420541,-111.930670
...,...,...,...,...,...
94,e8957e99332a66bd23650bc04b0c321e,Secret Warehouse,9886,33.441091,-112.082911
95,0cab138d1a9b55f141311a4a90e96d28,P168 5th St. and Washington,15,33.448519,-112.067322
96,d0c1a936f1125c6af6a566c4da5aeb82,ASU Business Administration Building (Virtual ...,5,33.416481,-111.933608
97,12e404d96fb464df02d1451942333ceb,Downtown ASU (Virtual Hub),2,33.453399,-112.072643


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [98]:
# Using dotenv to retrieve Foursquare API key
load_dotenv()
api_key_fs_string = 'FOURSQUARE_API_KEY' # replace with your Foursquare API key variable name
api_key_fs = os.getenv(api_key_fs_string)

In [270]:
# Function to find all POI's of a specific category within a set distance from any bike stations
def POIs(station_ids,latitudes,longitudes,radius,limit,category_ids):
    
    fields = 'fsq_id,name,rating,popularity,price'
    list_POI = []
    
    # Find all POI's within 1000m of each bike station.  This will return one column with station_id and one column containing lists of POI's.
    for stat_ids,lat,long in zip(station_ids,latitudes,longitudes):
        url = f'https://api.foursquare.com/v3/places/search?ll={lat},{long}&radius={radius}&categories={category_ids}&limit={limit}&fields={fields}'
        headers = {"accept": "application/json","Authorization": api_key_fs}
        response = requests.get(url, headers=headers).json()['results']
        list_POI.append([stat_ids,response])
   
    df = pd.DataFrame(list_POI,columns=['station_id','results']) # convert to dataframe
    df = df.explode('results').reset_index(drop=True) # create 1 row for each match of bike station and POI.  There are duplicates of both bike stations and POI's, but each matchup is unique.
    df = pd.concat([df,pd.json_normalize(df['results'])],axis=1).drop('results',axis=1)
    return df

In [261]:
# Category IDs: {Bars: 13003, Restaurant: 13065, Cafe Coffee and Tea House: 13032}

# Creating dataframe for bars/restaurants
df_retaurants_bars = POIs(station_ids=df_stations['id'],
                          latitudes=df_stations['latitude'],
                          longitudes=df_stations['longitude'],
                          radius=1000,
                          limit=50,
                          category_ids='13003,13065')

df_retaurants_bars

,station_id,fsq_id,name,popularity,price,rating
0,d334f65221f6a681f8b03433f36e2075,4ba42973f964a520e48738e3,Andrew's BBQ,0.927569,2.0,7.9
1,d334f65221f6a681f8b03433f36e2075,4ad5dea7f964a520cb0321e3,Bogeys Grill,0.984542,2.0,7.4
2,d334f65221f6a681f8b03433f36e2075,4bb13b11f964a5208d853ce3,Jack In The Box,0.985000,1.0,6.6
3,d334f65221f6a681f8b03433f36e2075,4c27e3463492a593369db628,Domino's Pizza,0.929312,1.0,6.5
4,d334f65221f6a681f8b03433f36e2075,4ba13047f964a52055a037e3,Chopstick Express,0.916712,1.0,6.6
...,...,...,...,...,...,...
3724,0e1960d94d8ab3ead0f26ddbd1cbf938,57574df3498e910a0863f9ed,Ike's Love & Sandwiches,0.966399,1.0,7.7
3725,0e1960d94d8ab3ead0f26ddbd1cbf938,413ba880f964a520b41b1fe3,Majerle's Sports Grill,0.982418,2.0,7.5
3726,0e1960d94d8ab3ead0f26ddbd1cbf938,466827eaf964a5208c471fe3,The Lost Leaf,0.955273,1.0,7.7
3727,0e1960d94d8ab3ead0f26ddbd1cbf938,58dee1d819b1ad32c1ac589e,Floor 13 Rooftop Bar,0.963778,2.0,7.1


In [262]:
# Creating dataframe for cafes, coffee shops, and tea houses
df_cafe_coffee_teaHouse = POIs(station_ids=df_stations['id'],
                          latitudes=df_stations['latitude'],
                          longitudes=df_stations['longitude'],
                          radius=1000,
                          limit=50,
                          category_ids='13032')

df_cafe_coffee_teaHouse

,station_id,fsq_id,name,popularity,price,rating
0,d334f65221f6a681f8b03433f36e2075,4ebd107d61af9815e0f894af,Dutch Bros Coffee,0.997571,1.0,9.0
1,cac663c57dee1dafd5f152bb906e97d5,4f67d02d003938727b2aa48d,Songbird Coffee & Tea House,0.964152,2.0,8.7
2,cac663c57dee1dafd5f152bb906e97d5,5808f6de38fa14bcdf99bb2f,Fillmore Coffee Co.,0.975837,1.0,7.9
3,cac663c57dee1dafd5f152bb906e97d5,5c9c8f58e0c0c9003967226c,Lacuna Kava Bar,0.948160,2.0,NaN
4,cac663c57dee1dafd5f152bb906e97d5,61d8ec72c77fa05364b3bd5a,"Jobot Coffee, Bar And Late Night Dim Sum",0.974346,1.0,NaN
...,...,...,...,...,...,...
1415,0e1960d94d8ab3ead0f26ddbd1cbf938,ecfacae8b3ce4936f4d6cd28,Fever Dream Coffee,NaN,NaN,NaN
1416,0e1960d94d8ab3ead0f26ddbd1cbf938,72e1884983dc49fc7e87680e,la Madeleine,NaN,NaN,NaN
1417,0e1960d94d8ab3ead0f26ddbd1cbf938,7d3d2b831c6845f921e8e337,Serafina Coffee Roasters,NaN,NaN,NaN
1418,0e1960d94d8ab3ead0f26ddbd1cbf938,036df707b7dd4940e862751a,Blue House Coffee Goods,NaN,NaN,NaN


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating

In [269]:
# There are 3 restaurants/bars tied for 1st in rating, and 8 tied for 2nd
df_retaurants_bars.sort_values('rating',ascending=False).drop_duplicates('name').head(11)

,station_id,fsq_id,name,popularity,price,rating
3180,4dbd1efcfc3dca0c0f204790a6d6bf51,4a5fcb9cf964a5203cc01fe3,Postino Central,0.981354,2.0,9.2
1061,9be3ff8f22f184a0781c655b409bfbe6,55600416498e5eb4dbecf791,Valley Bar,0.977446,2.0,9.2
3181,4dbd1efcfc3dca0c0f204790a6d6bf51,5085c381e4b038e4e19d739e,Federal Pizza,0.983153,2.0,9.2
2588,66e198c430281056829799daf8fb8a0a,4e3a4cdf1850f0453401b5fd,Crescent Ballroom,0.992263,1.0,9.1
1596,e8ea2c46e05597b2b23d1c3b0ced3e8d,4a3d9ccdf964a5208aa21fe3,In-N-Out Burger,0.999571,1.0,9.1
503,7139afb8e84dae818c1edeff867e735d,502ac807e4b02d4e93003f14,Angels Trumpet Ale House,0.976980,2.0,9.1
3177,4dbd1efcfc3dca0c0f204790a6d6bf51,413ba880f964a5206f1b1fe3,The Fry Bread House,0.981754,2.0,9.1
2231,d442d37ff6ef724e22dad67d3ca21925,413ba880f964a520ae1b1fe3,Lo-Lo's Chicken & Waffles,0.988618,2.0,9.1
518,7139afb8e84dae818c1edeff867e735d,4a3fdf4cf964a52004a41fe3,Cibo,0.977623,2.0,9.1
3131,ba402392fffdaea52a19f2c4846572d4,4e8a0dc45c5c370ff4894f15,Green New American Vegetarian,0.953506,1.0,9.1
